Olá, Gabriel!

Meu nome é Ramon. Fico feliz em revisar seu projeto. Ao ao longo do texto farei algumas observações sobre melhorias no código e também farei comentários sobre suas percepções sobre o assunto. Estarei aberto a feedbacks e discussões sobre o tema.

**Peço que mantenha e não altere os comentários que eu fizer por aqui para que possamos nos localizar posteriormente, ok?**

Mais uma coisa, vamos utilizar um código de cores para você entender os meus feedbacks no seu notebook. Funciona assim:

<div class="alert alert-block alert-success">
<b> Comentário do revisor: </b> <a class="tocSkip"></a>

Sucesso. Tudo foi feito corretamente.
</div>

<div class="alert alert-block alert-warning">
<b>Comentário do revisor: </b> <a class="tocSkip"></a>

Alerta não crítco, mas que pode ser corrigido para melhoria geral no seu código/análise
</div>

<div class="alert alert-block alert-danger">

<b>Comentário do revisor: </b> <a class="tocSkip"></a>
    
Erro que precisa ser consertado, caso contrário seu projeto **não** será aceito.
</div>

Você pode me responder usando isso:
<div class="alert alert-block alert-info">
<b>Resposta do Aluno.</b> <a class="tocSkip"></a>
</div>

# Desenvolvendo um modelo de aprendizado de máquina para prever o valor de mercado de um carro.

O serviço de vendas de carros usados Rusty Bargain está desenvolvendo um aplicativo para atrair novos clientes. Nesse aplicativo, você pode descobrir rapidamente o valor de mercado do seu carro. Você tem acesso a dados históricos: especificações técnicas, versões de acabamento e preços. Você precisa construir o modelo para determinar o valor. 

Rusty Bargain está interessado em:

- a qualidade da predição;
- a velocidade da predição;
- o tempo necessário para o treinamento

## Preparação de Dados

### Iniciação
Carregue bibliotecas que você acredita que são necessárias para o projeto. Você pode perceber depois que você precisa de bibliotecas adicionais conforme você avança, o que é totalmente normal - apenas certifique-se de atualizar essa seção quando perceber isso.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats as st
import warnings
warnings.filterwarnings("ignore")

In [2]:
# Carregue o arquivo com os dados em um DataFrame
df = pd.read_csv('/datasets/car_data.csv')

### Explore os dados iniciais



#### Descrição de dados

- `DateCrawled` — data em que o perfil foi baixado do banco de dados
- `VehicleType` — tipo de carroçaria do veículo
- `RegistrationYear` — ano de matrícula do veículo
- `Gearbox` — tipo de caixa de transmissão
- `Power` — potência (hp)
- `Model` — modelo do veículo
- `Mileage` — quilometragem (medida em km devido às especificidades regionais do conjunto de dados)
- `RegistrationMonth` — mês de registro do veículo
- `FuelType` — tipo de combustível
- `Brand` — marca do veículo
- `NotRepaired` — veículo reparado ou não
- `DateCreated` — data de criação do perfil
- `NumberOfPictures` — número de fotos do veículo
- `PostalCode` — código postal do proprietário do perfil (usuário)
- `LastSeen` — data da última atividade do usuário

Objetivo

- `Price` — preço (Euro)

Agora vamos explorar nossos dados. Você vai querer ver quantas colunas e linhas ele tem, veja algumas linhas para verificar possíveis problemas com os dados.

In [3]:
# vamos exibir as primeiras 5 linhas
df.head(5)

,DateCrawled,Price,VehicleType,RegistrationYear,Gearbox,Power,Model,Mileage,RegistrationMonth,FuelType,Brand,NotRepaired,DateCreated,NumberOfPictures,PostalCode,LastSeen
0,24/03/2016 11:52,480,NaN,1993,manual,0,golf,150000,0,petrol,volkswagen,NaN,24/03/2016 00:00,0,70435,07/04/2016 03:16
1,24/03/2016 10:58,18300,coupe,2011,manual,190,NaN,125000,5,gasoline,audi,yes,24/03/2016 00:00,0,66954,07/04/2016 01:46
2,14/03/2016 12:52,9800,suv,2004,auto,163,grand,125000,8,gasoline,jeep,NaN,14/03/2016 00:00,0,90480,05/04/2016 12:47
3,17/03/2016 16:54,1500,small,2001,manual,75,golf,150000,6,petrol,volkswagen,no,17/03/2016 00:00,0,91074,17/03/2016 17:40
4,31/03/2016 17:25,3600,small,2008,manual,69,fabia,90000,7,gasoline,skoda,no,31/03/2016 00:00,0,60437,06/04/2016 10:17


In [4]:
# Vamos ver quantas linhas e colunas nosso conjunto de dados tem
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 354369 entries, 0 to 354368
Data columns (total 16 columns):
 #   Column             Non-Null Count   Dtype 
---  ------             --------------   ----- 
 0   DateCrawled        354369 non-null  object
 1   Price              354369 non-null  int64 
 2   VehicleType        316879 non-null  object
 3   RegistrationYear   354369 non-null  int64 
 4   Gearbox            334536 non-null  object
 5   Power              354369 non-null  int64 
 6   Model              334664 non-null  object
 7   Mileage            354369 non-null  int64 
 8   RegistrationMonth  354369 non-null  int64 
 9   FuelType           321474 non-null  object
 10  Brand              354369 non-null  object
 11  NotRepaired        283215 non-null  object
 12  DateCreated        354369 non-null  object
 13  NumberOfPictures   354369 non-null  int64 
 14  PostalCode         354369 non-null  int64 
 15  LastSeen           354369 non-null  object
dtypes: int64(7), object(

In [5]:
# Verificar Valores Duplicados
df.duplicated().sum()

262

In [6]:
# Aborde as duplicatas, se existirem
df = df.drop_duplicates().reset_index(drop=True)

In [7]:
# Verificar Valores Duplicados
df.duplicated().sum()

0

In [8]:
# Verificar Valores Ausentes
df.isna().sum()

DateCrawled              0
Price                    0
VehicleType          37484
RegistrationYear         0
Gearbox              19830
Power                    0
Model                19701
Mileage                  0
RegistrationMonth        0
FuelType             32889
Brand                    0
NotRepaired          71145
DateCreated              0
NumberOfPictures         0
PostalCode               0
LastSeen                 0
dtype: int64

In [9]:
# Vamos calcular a porcentagem desses valores faltantes em relação com o nosso conjunto de dados
(df.isnull().sum() / df.shape[0]) * 100

DateCrawled           0.000000
Price                 0.000000
VehicleType          10.585501
RegistrationYear      0.000000
Gearbox               5.600002
Power                 0.000000
Model                 5.563573
Mileage               0.000000
RegistrationMonth     0.000000
FuelType              9.287871
Brand                 0.000000
NotRepaired          20.091385
DateCreated           0.000000
NumberOfPictures      0.000000
PostalCode            0.000000
LastSeen              0.000000
dtype: float64

Removemos os dados duplicados e analizamos os valores faltantes. 

### Transformação de dados
Vamos examinar cada coluna para ver quais problemas podemos ter nelas.

In [11]:
df['DateCrawled'].sort_values()

44600     01/04/2016 00:06
266156    01/04/2016 00:06
157525    01/04/2016 00:06
119430    01/04/2016 00:10
122927    01/04/2016 00:25
                ...       
78331     31/03/2016 23:58
344885    31/03/2016 23:58
102481    31/03/2016 23:58
59088     31/03/2016 23:58
201452    31/03/2016 23:58
Name: DateCrawled, Length: 354107, dtype: object

In [12]:
df['Price'].describe()

count    354107.000000
mean       4416.433287
std        4514.338584
min           0.000000
25%        1050.000000
50%        2700.000000
75%        6400.000000
max       20000.000000
Name: Price, dtype: float64

In [13]:
df['VehicleType'].sort_values().unique()

array(['bus', 'convertible', 'coupe', 'other', 'sedan', 'small', 'suv',
       'wagon', nan], dtype=object)

In [14]:
df['RegistrationYear'].sort_values().unique()

array([1000, 1001, 1039, 1111, 1200, 1234, 1253, 1255, 1300, 1400, 1500,
       1600, 1602, 1688, 1800, 1910, 1915, 1919, 1920, 1923, 1925, 1927,
       1928, 1929, 1930, 1931, 1932, 1933, 1934, 1935, 1936, 1937, 1938,
       1940, 1941, 1942, 1943, 1944, 1945, 1946, 1947, 1948, 1949, 1950,
       1951, 1952, 1953, 1954, 1955, 1956, 1957, 1958, 1959, 1960, 1961,
       1962, 1963, 1964, 1965, 1966, 1967, 1968, 1969, 1970, 1971, 1972,
       1973, 1974, 1975, 1976, 1977, 1978, 1979, 1980, 1981, 1982, 1983,
       1984, 1985, 1986, 1987, 1988, 1989, 1990, 1991, 1992, 1993, 1994,
       1995, 1996, 1997, 1998, 1999, 2000, 2001, 2002, 2003, 2004, 2005,
       2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016,
       2017, 2018, 2019, 2066, 2200, 2222, 2290, 2500, 2800, 2900, 3000,
       3200, 3500, 3700, 3800, 4000, 4100, 4500, 4800, 5000, 5300, 5555,
       5600, 5900, 5911, 6000, 6500, 7000, 7100, 7500, 7800, 8000, 8200,
       8455, 8500, 8888, 9000, 9229, 9450, 9996, 99

In [15]:
df['Gearbox'].sort_values().unique()

array(['auto', 'manual', nan], dtype=object)

In [16]:
df['Power'].describe()

count    354107.000000
mean        110.089651
std         189.914972
min           0.000000
25%          69.000000
50%         105.000000
75%         143.000000
max       20000.000000
Name: Power, dtype: float64

In [17]:
df['Model'].sort_values().unique()

array(['100', '145', '147', '156', '159', '1_reihe', '1er', '200',
       '2_reihe', '300c', '3_reihe', '3er', '4_reihe', '500', '5_reihe',
       '5er', '601', '6_reihe', '6er', '7er', '80', '850', '90', '900',
       '9000', '911', 'a1', 'a2', 'a3', 'a4', 'a5', 'a6', 'a8',
       'a_klasse', 'accord', 'agila', 'alhambra', 'almera', 'altea',
       'amarok', 'antara', 'arosa', 'astra', 'auris', 'avensis', 'aveo',
       'aygo', 'b_klasse', 'b_max', 'beetle', 'berlingo', 'bora',
       'boxster', 'bravo', 'c1', 'c2', 'c3', 'c4', 'c5', 'c_klasse',
       'c_max', 'c_reihe', 'caddy', 'calibra', 'captiva', 'carisma',
       'carnival', 'cayenne', 'cc', 'ceed', 'charade', 'cherokee',
       'citigo', 'civic', 'cl', 'clio', 'clk', 'clubman', 'colt', 'combo',
       'cooper', 'cordoba', 'corolla', 'corsa', 'cr_reihe', 'croma',
       'crossfire', 'cuore', 'cx_reihe', 'defender', 'delta', 'discovery',
       'doblo', 'ducato', 'duster', 'e_klasse', 'elefantino', 'eos',
       'escort', 'espac

In [18]:
df['Mileage'].describe()

count    354107.000000
mean     128211.811684
std       37906.590101
min        5000.000000
25%      125000.000000
50%      150000.000000
75%      150000.000000
max      150000.000000
Name: Mileage, dtype: float64

In [19]:
df['RegistrationMonth'].sort_values().unique()

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12])

In [20]:
df['FuelType'].sort_values().unique()

array(['cng', 'electric', 'gasoline', 'hybrid', 'lpg', 'other', 'petrol',
       nan], dtype=object)

In [21]:
df['Brand'].sort_values().unique()

array(['alfa_romeo', 'audi', 'bmw', 'chevrolet', 'chrysler', 'citroen',
       'dacia', 'daewoo', 'daihatsu', 'fiat', 'ford', 'honda', 'hyundai',
       'jaguar', 'jeep', 'kia', 'lada', 'lancia', 'land_rover', 'mazda',
       'mercedes_benz', 'mini', 'mitsubishi', 'nissan', 'opel', 'peugeot',
       'porsche', 'renault', 'rover', 'saab', 'seat', 'skoda', 'smart',
       'sonstige_autos', 'subaru', 'suzuki', 'toyota', 'trabant',
       'volkswagen', 'volvo'], dtype=object)

In [22]:
df['NotRepaired'].sort_values().unique()

array(['no', 'yes', nan], dtype=object)

In [23]:
df['DateCreated'].sort_values()

247165    01/02/2016 00:00
293735    01/02/2016 00:00
281833    01/02/2016 00:00
201856    01/03/2016 00:00
296916    01/03/2016 00:00
                ...       
319455    31/03/2016 00:00
55814     31/03/2016 00:00
185358    31/03/2016 00:00
185419    31/03/2016 00:00
218578    31/03/2016 00:00
Name: DateCreated, Length: 354107, dtype: object

In [24]:
df['NumberOfPictures'].describe()

count    354107.0
mean          0.0
std           0.0
min           0.0
25%           0.0
50%           0.0
75%           0.0
max           0.0
Name: NumberOfPictures, dtype: float64

In [25]:
df['PostalCode'].sort_values().unique()

array([ 1067,  1068,  1069, ..., 99994, 99996, 99998])

In [26]:
df['LastSeen'].sort_values()

311229    01/04/2016 00:15
300128    01/04/2016 00:15
270029    01/04/2016 00:15
84894     01/04/2016 00:15
327905    01/04/2016 00:15
                ...       
15495     31/03/2016 23:47
231071    31/03/2016 23:48
36469     31/03/2016 23:50
262530    31/03/2016 23:51
72795     31/03/2016 23:54
Name: LastSeen, Length: 354107, dtype: object

As colunas 'PostalCode' , 'LastSeen' , 'DateCrawled' e 'DateCreated', possuem valorem que não servem para nossa pesquisa por isso iremos retira-las. As colunas 'price' e 'power', possui alguns valores anormais, nos iremos retirar esses valores

### Pré-processar dados 

In [28]:
# a lista dos nomes das colunas na tabela df
df.columns

Index(['DateCrawled', 'Price', 'VehicleType', 'RegistrationYear', 'Gearbox',
       'Power', 'Model', 'Mileage', 'RegistrationMonth', 'FuelType', 'Brand',
       'NotRepaired', 'DateCreated', 'NumberOfPictures', 'PostalCode',
       'LastSeen'],
      dtype='object')

In [29]:
# renomeando colunas
df = df.rename(str.lower, axis='columns')

In [30]:
# a lista dos nomes das colunas na tabela df
df.columns

Index(['datecrawled', 'price', 'vehicletype', 'registrationyear', 'gearbox',
       'power', 'model', 'mileage', 'registrationmonth', 'fueltype', 'brand',
       'notrepaired', 'datecreated', 'numberofpictures', 'postalcode',
       'lastseen'],
      dtype='object')

In [32]:
#Deixando as colunas que realmentes influenciam nossa pesquisa
df = df[['price','vehicletype','registrationyear','gearbox',
       'power', 'model', 'mileage', 'registrationmonth', 'fueltype', 'brand',
       'notrepaired']]

In [33]:
#Retirando valores do Mês que correspondem a 0
df = df.query("registrationmonth > 0")

In [34]:
#Subistituindo valores NAN de 'notrepaired' para 'unknownrepaired'
df['notrepaired'].fillna('unknownrepaired',inplace=True )

In [35]:
#Retirando outliers da coluna 'price' e power
Q1_price = df['price'].quantile(0.25)
Q3_price =df['price'].quantile(0.75)
IQR_price = Q3_price - Q1_price
limite_superior_price = Q3_price+(1.5*IQR_price)
limite_inferior_price = Q1_price - (1.5 * IQR_price)

In [36]:
Q1_power = df['power'].quantile(0.25)
Q3_power =df['power'].quantile(0.75)
IQR_power = Q3_power - Q1_power
limite_superior_power = Q3_power+(1.5*IQR_power)
limite_inferior_power = Q1_power - (1.5 * IQR_power)


In [37]:
df_final = df[df['price'] > limite_inferior_price] 
df_final = df_final[df_final['price'] < limite_superior_price]
df_final = df_final[df_final['power'] > limite_inferior_power] 
df_final = df_final[df_final['power'] < limite_superior_power]

In [38]:
df_final['price'].describe()

count    296234.000000
mean       4016.808277
std        3648.408706
min           0.000000
25%        1200.000000
50%        2790.000000
75%        5950.000000
max       15249.000000
Name: price, dtype: float64

In [39]:
#valores abaixo do outliers sobstituindo com a média em preço
dfpriceok = df_final.query("price > @Q1_price ")
dfpriceok = dfpriceok.groupby(['vehicletype','registrationyear','gearbox','model'])['price'].mean()

In [40]:
#valores abaixo do outliers sobstituindo com a média em potência
dfpowerok = df_final.query("power > @Q1_power ")
dfpowerok = dfpowerok.groupby(['vehicletype','registrationyear','gearbox','model'])['price'].mean()

In [41]:
def fill_price(row):
    if row['price'] <  Q1_price :
        return dfpriceok[row['price']]
    return row['price']


In [42]:
def fill_power(row):
    if row['power'] <  Q1_power :
        return dfpowerok[row['power']]
    return row['power']

<div class="alert alert-block alert-success">
<b> Comentário do revisor: </b> <a class="tocSkip"></a>

Correto. Uma estratégai mais rebuscada é levar em consideração o ano ou outras variáveis
</div>

In [43]:
df_final['price'] = df_final.apply(fill_price, axis=1)
df_final['power'] = df_final.apply(fill_power, axis=1)

In [44]:
df_final['price'].describe()

count    296234.000000
mean       5690.894652
std        3647.273691
min        1250.000000
25%        2500.000000
50%        4812.500000
75%        7999.000000
max       15249.000000
Name: price, dtype: float64

In [45]:
df_final['power'].describe()

count    296234.000000
mean       1258.166519
std        2429.866596
min          75.000000
25%         105.000000
50%         140.000000
75%         606.750000
max       12400.000000
Name: power, dtype: float64

In [46]:
(df_final.isnull().sum() / df_final.shape[0]) * 100

price                0.000000
vehicletype          7.405632
registrationyear     0.000000
gearbox              3.319335
power                0.000000
model                4.146384
mileage              0.000000
registrationmonth    0.000000
fueltype             6.270718
brand                0.000000
notrepaired          0.000000
dtype: float64

In [47]:
#como os valores dos valores faltantes são umastra pequena de todos os dados, por isso estão retirando
df_final.dropna(inplace=True)

In [48]:
(df_final.isnull().sum() / df_final.shape[0]) * 100

price                0.0
vehicletype          0.0
registrationyear     0.0
gearbox              0.0
power                0.0
model                0.0
mileage              0.0
registrationmonth    0.0
fueltype             0.0
brand                0.0
notrepaired          0.0
dtype: float64

In [49]:
df_final.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 251133 entries, 2 to 354106
Data columns (total 11 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   price              251133 non-null  float64
 1   vehicletype        251133 non-null  object 
 2   registrationyear   251133 non-null  int64  
 3   gearbox            251133 non-null  object 
 4   power              251133 non-null  float64
 5   model              251133 non-null  object 
 6   mileage            251133 non-null  int64  
 7   registrationmonth  251133 non-null  int64  
 8   fueltype           251133 non-null  object 
 9   brand              251133 non-null  object 
 10  notrepaired        251133 non-null  object 
dtypes: float64(2), int64(3), object(6)
memory usage: 23.0+ MB


In [50]:
df_final

,price,vehicletype,registrationyear,gearbox,power,model,mileage,registrationmonth,fueltype,brand,notrepaired
2,9800.0,suv,2004,auto,163.000000,grand,125000,8,gasoline,jeep,unknownrepaired
3,1500.0,small,2001,manual,75.000000,golf,150000,6,petrol,volkswagen,no
4,3600.0,small,2008,manual,2566.333333,fabia,90000,7,gasoline,skoda,no
5,4900.0,sedan,1995,manual,102.000000,3er,150000,10,petrol,bmw,yes
6,2200.0,convertible,2004,manual,109.000000,2_reihe,150000,8,petrol,peugeot,no
...,...,...,...,...,...,...,...,...,...,...,...
354100,3200.0,sedan,2004,manual,225.000000,leon,150000,5,petrol,seat,yes
354101,14990.0,bus,2000,manual,8000.000000,zafira,150000,3,petrol,opel,no
354104,12950.0,convertible,2000,auto,101.000000,fortwo,125000,3,petrol,smart,no
354105,9200.0,bus,1996,manual,102.000000,transporter,150000,3,gasoline,volkswagen,no


In [51]:
(df_final.isnull().sum() / df_final.shape[0]) * 100

price                0.0
vehicletype          0.0
registrationyear     0.0
gearbox              0.0
power                0.0
model                0.0
mileage              0.0
registrationmonth    0.0
fueltype             0.0
brand                0.0
notrepaired          0.0
dtype: float64

In [52]:
df_final = df_final.reset_index(drop=True)

In [53]:
df_final

,price,vehicletype,registrationyear,gearbox,power,model,mileage,registrationmonth,fueltype,brand,notrepaired
0,9800.0,suv,2004,auto,163.000000,grand,125000,8,gasoline,jeep,unknownrepaired
1,1500.0,small,2001,manual,75.000000,golf,150000,6,petrol,volkswagen,no
2,3600.0,small,2008,manual,2566.333333,fabia,90000,7,gasoline,skoda,no
3,4900.0,sedan,1995,manual,102.000000,3er,150000,10,petrol,bmw,yes
4,2200.0,convertible,2004,manual,109.000000,2_reihe,150000,8,petrol,peugeot,no
...,...,...,...,...,...,...,...,...,...,...,...
251128,3200.0,sedan,2004,manual,225.000000,leon,150000,5,petrol,seat,yes
251129,14990.0,bus,2000,manual,8000.000000,zafira,150000,3,petrol,opel,no
251130,12950.0,convertible,2000,auto,101.000000,fortwo,125000,3,petrol,smart,no
251131,9200.0,bus,1996,manual,102.000000,transporter,150000,3,gasoline,volkswagen,no


## Treinamento do modelo

In [54]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import OrdinalEncoder

In [55]:
#Codificação de valores categóricos
object_columns = df_final.select_dtypes(include='object')
encoder = OrdinalEncoder()
codification = pd.DataFrame(encoder.fit_transform(object_columns), columns=object_columns.columns)
df_final[object_columns.columns] = codification

In [56]:
df_train_valid, df_test = train_test_split(df_final, test_size=0.2)
df_train, df_valid = train_test_split(df_train_valid, test_size=0.25)

In [57]:
features_train = df_train.drop(['price'], axis=1)
target_train = df_train['price']
features_valid = df_valid.drop(['price'], axis=1)
target_valid = df_valid['price']
features_test = df_test.drop(['price'], axis=1)
target_test = df_test['price']

### LightGBM

In [91]:
import timeit
from sklearn.model_selection import GridSearchCV
import lightgbm as lgb

In [92]:
params = {
    'task': 'train', 
    'boosting': 'gbdt',
    'objective': 'regression',
    'metric': 'root_mean_squared_error',
    'num_leaves': 10,
    'learning_rate':0.05,
    'feature_fraction':0.5,
    'random_state': 12345,
    'verbose':-1
}

In [93]:
lgb_train = lgb.Dataset(features_train, target_train)
lgb_eval = lgb.Dataset(features_valid, target_valid, reference=lgb_train)

In [94]:
modellightgbm = lgb.train(params,
                 train_set=lgb_train,
                 valid_sets=lgb_eval,
                 early_stopping_rounds=30)

[1]	valid_0's rmse: 3625.23
Training until validation scores don't improve for 30 rounds
[2]	valid_0's rmse: 3610.49
[3]	valid_0's rmse: 3596.97
[4]	valid_0's rmse: 3560.11
[5]	valid_0's rmse: 3535.26
[6]	valid_0's rmse: 3511.14
[7]	valid_0's rmse: 3505.36
[8]	valid_0's rmse: 3471.25
[9]	valid_0's rmse: 3456.24
[10]	valid_0's rmse: 3419.38
[11]	valid_0's rmse: 3388.56
[12]	valid_0's rmse: 3384.49
[13]	valid_0's rmse: 3353.44
[14]	valid_0's rmse: 3345.99
[15]	valid_0's rmse: 3321.44
[16]	valid_0's rmse: 3312.56
[17]	valid_0's rmse: 3289.66
[18]	valid_0's rmse: 3264.84
[19]	valid_0's rmse: 3258.51
[20]	valid_0's rmse: 3235
[21]	valid_0's rmse: 3231.78
[22]	valid_0's rmse: 3218.4
[23]	valid_0's rmse: 3201.98
[24]	valid_0's rmse: 3195.72
[25]	valid_0's rmse: 3192.99
[26]	valid_0's rmse: 3176.58
[27]	valid_0's rmse: 3157.8
[28]	valid_0's rmse: 3148.58
[29]	valid_0's rmse: 3133.5
[30]	valid_0's rmse: 3128.92
[31]	valid_0's rmse: 3114.66
[32]	valid_0's rmse: 3107.36
[33]	valid_0's rmse: 3095.

In [95]:
predictions_valid_lgbm = modellightgbm.predict(features_valid)

In [61]:
import math
from sklearn.metrics import mean_squared_error as mse

In [96]:
math.sqrt(mse(target_valid, predictions_valid_lgbm))

2891.3403073908544

In [63]:
%%timeit
# Tempo de predição do modelo Árvore de Decisão
modellightgbm.predict(features_train)

1.16 s ± 40 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


###  Árvore de Decisão


In [64]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error as mse

In [65]:
# Modelagem da Árvore de Decisão buscando o melhor hiperparâmetro.
best_model_ad = None
best_result = 10000
best_depth = 0
best_time = 0
for depth in range(1, 50):
    model_dcre = DecisionTreeRegressor(random_state=12345, max_depth=depth)
    model_dcre.fit(features_train, target_train) 
    predictions_valid_dcre = model_dcre.predict(features_valid)
    result = math.sqrt(mse(target_valid, predictions_valid_dcre))
    if result < best_result:
        best_model_dcre = model_dcre
        best_result = result
        best_depth = depth
        

print(f"O melhor resultado REQM da Árvore de Decisão é com o hiperparâmetro max_depth de {best_depth}: {best_result}")

O melhor resultado REQM da Árvore de Decisão é com o hiperparâmetro max_depth de 11: 2813.075956370124


In [66]:
%%timeit
# Tempo de predição do modelo Árvore de Decisão
best_model_dcre.predict(features_train)

25.8 ms ± 964 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


### Floresta Aleatória

In [67]:
from sklearn.ensemble import RandomForestRegressor

In [68]:
# Modelagem da Floresta Aleatória buscando o melhor hiperparâmetro.
best_model = None
best_result = 10000
best_est = 0
best_depth = 0
for est in range(40, 51, 5):
    for depth in range (10, 15, 5):
        model_fare = RandomForestRegressor(random_state=12345, n_estimators=est, max_depth=depth)
        model_fare.fit(features_train, target_train)
        predictions_valid_fare = model_fare.predict(features_valid)
        result = math.sqrt(mse(target_valid, predictions_valid_fare))
        if result < best_result:
            best_model_fare = model_fare
            best_result = result
            best_est = est
            best_depth = depth

print("O melhor resultado de REQM da Floresta Aleatória é de:", best_result, ", com o hiperparâmetro n_estimators de:", best_est, "e melhor depth de:", depth)

O melhor resultado de REQM da Floresta Aleatória é de: 2757.7108835398553 , com o hiperparâmetro n_estimators de: 40 e melhor depth de: 10


<div class="alert alert-block alert-success">
<b> Comentário do revisor: </b> <a class="tocSkip"></a>

Correto. Em geral, utilizamos numero maior de estimadores. Profundidade de 15 é geralmente elevada. Pense que em cada nível h há 2^h nós se a árvore for binária e completa. Isso pode resultar em overfitting.
</div>

In [69]:
%%timeit
# Tempo de execução do modelo Floresta Aleatória
best_model_fare.predict(features_train)

692 ms ± 14.3 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


### Regressão Linear

In [70]:
from sklearn.linear_model import LinearRegression

In [71]:
model_lr = LinearRegression()
model_lr.fit(features_train, target_train)
predictions_valid_lr = model_lr.predict(features_valid)

In [72]:
math.sqrt(mse(target_valid, predictions_valid_lr))

3432.636784367167

In [73]:
%%timeit
# Tempo de execução do modelo Floresta Aleatória
model_lr.predict(features_train)


21.7 ms ± 2.66 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [76]:
from xgboost import XGBRegressor 
from sklearn.pipeline import Pipeline

In [77]:
xgb_model  = XGBRegressor(nthread=10)
clf = Pipeline([('xgb', xgb_model)])
param_grid = {'xgb__max_depth': [1,2,3],    
              'xgb__learning_rate': [0.2,0.3,0.4],  
              'xgb__n_estimators': [10,15,5]   
             }  

model = GridSearchCV(estimator=clf, param_grid=param_grid, verbose=10, n_jobs=1, refit=True, cv=10)

model.fit(features_train, target_train)

Fitting 10 folds for each of 27 candidates, totalling 270 fits
[CV 1/10; 1/27] START xgb__learning_rate=0.2, xgb__max_depth=1, xgb__n_estimators=10
[CV 1/10; 1/27] END xgb__learning_rate=0.2, xgb__max_depth=1, xgb__n_estimators=10; total time=   0.5s
[CV 2/10; 1/27] START xgb__learning_rate=0.2, xgb__max_depth=1, xgb__n_estimators=10
[CV 2/10; 1/27] END xgb__learning_rate=0.2, xgb__max_depth=1, xgb__n_estimators=10; total time=   0.5s
[CV 3/10; 1/27] START xgb__learning_rate=0.2, xgb__max_depth=1, xgb__n_estimators=10
[CV 3/10; 1/27] END xgb__learning_rate=0.2, xgb__max_depth=1, xgb__n_estimators=10; total time=   0.4s
[CV 4/10; 1/27] START xgb__learning_rate=0.2, xgb__max_depth=1, xgb__n_estimators=10
[CV 4/10; 1/27] END xgb__learning_rate=0.2, xgb__max_depth=1, xgb__n_estimators=10; total time=   0.5s
[CV 5/10; 1/27] START xgb__learning_rate=0.2, xgb__max_depth=1, xgb__n_estimators=10
[CV 5/10; 1/27] END xgb__learning_rate=0.2, xgb__max_depth=1, xgb__n_estimators=10; total time=   0.

GridSearchCV(cv=10,
             estimator=Pipeline(steps=[('xgb',
                                        XGBRegressor(base_score=None,
                                                     booster=None,
                                                     colsample_bylevel=None,
                                                     colsample_bynode=None,
                                                     colsample_bytree=None,
                                                     enable_categorical=False,
                                                     gamma=None, gpu_id=None,
                                                     importance_type=None,
                                                     interaction_constraints=None,
                                                     learning_rate=None,
                                                     max_delta_step=None,
                                                     max_depth=None,
                                    

In [79]:
predictions_valid_xgb = model.predict(features_valid)

In [81]:
math.sqrt(mse(target_valid, predictions_valid_xgb))

2904.1016535395875

In [82]:
%%timeit
# Tempo de predição do modelo Árvore de Decisão
model.predict(features_train)

106 ms ± 3.94 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


## Conclusão

Neste projeto, importamos bibliotecas como "pandas" e "numpy" que habitualmente já utilizamos e sklearn para criação do nosso modelo.

Realizamos o pre-processamento, para que não tivesse acontecido algum problema na criação do nosso modelo. O modelo com melhor pontuação no **REQM** foi o Modelo Floresta Aletatória, enquanto o com pior desempenho foi o de Regressão Linear, Testamos o modelo de Árvore de Decisão e XG Boost.


<div class="alert alert-block alert-success">
<b>Comentário Geral do Revisor</b> <a class="tocSkip"></a>

Obrigado por enviar seu projeto. 
    
Parabéns, você fez um bom trabalho. No geral, você respondeu corretamente as questões.

Como pontos fortes do projeto, destaco:
- Código simples e eficiente
- Analisou outliers, realizou imputação
- Calibrou hiperparâmetros

Desejo sucesso na jornada.
Conte conosco. Lembre-se de que você pode tirar dúvidas comigo usando os comentários ou buscar ajuda de um tutor.

</div>

# Checklist

Digite 'x' para verificar. Em seguida, pressione Shift + Enter.

- [x]  O Jupyter Notebook está aberto
- [ ]  O código está livre de erros
- [ ]  As células com o código foram organizadas em ordem de execução
- [ ]  Os dados foram baixados e preparados
- [ ]  Os modelos foram treinados
- [ ]  A análise de velocidade e qualidade dos modelos foi realizada